## Importación de librerías y montado del drive (es necesario dar permiso en las ventanas emergentes)

### Se creó un pickle que contiene el Oauth 2.0 para conectar con google drive. 
***Asegúrate de que el auth.json en tu caso es correcto***


In [24]:
import pandas as pd
from tqdm import tqdm
import requests
import ast
from sklearn import metrics
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')

## Datos a utilizar (deben ser reemplazados por los que corresponda)

In [26]:
# Funciones de manejo de token (pickle)
def save_token(token, token_path='token.pickle'):
    """Guarda el token o cualquier objeto en un archivo pickle."""
    with open(token_path, 'wb') as token_file:
        pickle.dump(token, token_file)
    print(f"Token guardado en {token_path}")

def load_token(token_path='token.pickle'):
    """Carga el token desde un archivo pickle."""
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token_file:
            token = pickle.load(token_file)
        print(f"Token cargado desde {token_path}")
        return token
    else:
        print("No se encontró un archivo pickle. Retornando None.")
        return None

In [ ]:
# Nombre del dataset exacto guardado en el directorio 
NOMBRE_PROYECTO = ""

# Se recomienda no cambiar estos directorios, donde se deja el csv inferido
DATASETS = "/content/drive/Externo/XBOX/"
OUTPUT = "/content/drive/Outputs"

# Entidades a dirigir la reputación
TARGET_ENTITIES = ['tigre de zucaritas','zucaritas','tigre.?toño','toño']

## Invocación de la api del Modelo GEA (IA de LLYC)

In [ ]:
print('Procesando...')

df = pd.read_csv(DATASETS + NOMBRE_PROYECTO, skiprows=6)
df['Target Entities'] = df.apply(lambda row: TARGET_ENTITIES.copy(), axis=1)

# El análisis de 10.000 filas puede llevar en torno a 1 minuto.
lang = 'es' # Admite es o pt
data = {"language": lang, "urls": df['Url'].tolist(), 'tweets': df['Full Text'].tolist(), 'target_entities': df['Target Entities'].tolist()}

r = requests.post("https://gea.llyc.app/gea2/", json=data, timeout=500)
if r.status_code == 200:
    print('Inferencia terminada!')
else:
    raise Exception(r.text)

## Recupero la inferencia

In [ ]:
df['tweet_prepro'] = r.json()['tweets']
df['ia_llyc_sentiment'] = r.json()['preds']

df.to_csv(OUTPUT + NOMBRE_PROYECTO[:-4], index= False)
print ("Dataset {} inferido en directorio output!".format(NOMBRE_PROYECTO[:-4]))

# TESTADOR DE FRASES SUELTAS

In [ ]:
# Prueba de que la llamada a la API con el análisis de sentimiento de una frase
def inf_tweet (message, target_entities):
        print("Mensaje:  ", message)
        print("Entidades:", target_entities, end='\n\n')

        data = {"language": 'es', "urls": ['-sin url-'], 'tweets': [message], 'target_entities': [target_entities]}

        r = requests.post("https://gea.llyc.app/gea2/", json=data, timeout=60)
        if r.status_code == 200:
            print(r.json())
        else:
            raise Exception(r.text)

In [ ]:
inf_tweet('Pepsi como refresco está rico y es mucho mejor que Trina', ['pepsi cola','pepsicola','pepsi'])

In [ ]:
inf_tweet('Pepsi como refresco está rico y es mucho mejor que Trina', ['trina','trinaranjus'])